In [250]:
import json
from datetime import datetime 
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
from os import listdir
from os.path import isfile, join

global log_list
global log_print
log_print = False
log_list = []

In [251]:
def logit(logstr):
    global log_list
    timestamp = '[' + datetime.now().strftime("%Y/%m/%d %H:%M:%S") + ']'
    logstr = timestamp + ' ' + logstr
    log_list.append(logstr)
    if log_print:
        print(logstr)

In [252]:
mask_class = {'face_with_mask':0,'face_no_mask':1,'face_with_mask_incorrect':2}
root_path = 'C:\\Users\\jyliao\\AIA\\'
output_path = root_path + 'output\\'
anno_path = root_path + 'annotations\\'
pic_path = root_path + 'images\\'

path_list = [root_path,output_path,anno_path,pic_path]

logit('Define root_path -> ' + root_path)
logit('Define output_path -> ' + output_path)
logit('Define anno_path -> ' + anno_path)
logit('Define pic_path -> ' + pic_path)

In [253]:
CHECK_FOLDER = os.path.isdir(output_path)
if not CHECK_FOLDER:
    os.makedirs(output_path)
    logit("created folder : " + output_path)
else:
    logit(output_path + " folder already exists.")

In [254]:
folder_check = True

for folder_path in path_list:
    CHECK_FOLDER = os.path.isdir(folder_path)
    if not CHECK_FOLDER:
        logit(folder_path + ' missing !')
        folder_check = False
    else:
        logit(folder_path + " check ok!")

In [255]:
if folder_check:
    logit('Start anno for each loop.')

    anno_file_list = [f for f in listdir(anno_path) if isfile(join(anno_path, f))]
    pic_file_list = [f for f in listdir(pic_path) if isfile(join(pic_path, f))]
    logit('Anno files count :' + str(len(anno_file_list)))
    logit('Pic files count :' + str(len(pic_file_list)))

    success_count = 0
    fail_count = 0

    for anno in anno_file_list:
        yolo_str_list = []
        try:
            with open(anno_path + anno, newline='') as jsonfile:
                jsonStr = jsonfile
                data = json.load(jsonfile)

            if data['FileName'] in pic_file_list:
                im = Image.open(pic_path + data['FileName'])
                ori_width, ori_height = im.size
                del im

                for anno in data['Annotations']:                
                    if anno['classname'] in list(mask_class.keys()):
                        bbox = anno['BoundingBox']
                        width = (bbox[2] - bbox[0])
                        height = (bbox[3] - bbox[1])
                        centerX = int((width/2) + bbox[0])
                        centerY = int((height/2) + bbox[1])

                        yolo_cx,yolo_cy,yolo_w,yolo_h = centerX/ori_width,centerY/ori_height,width/ori_width,height/ori_height
                        yolo_label=[mask_class[anno['classname']],yolo_cx,yolo_cy,yolo_w,yolo_h]
                        yolo_label = [str(x) for x in yolo_label]
                        yolo_label_str = ' '.join(yolo_label)
                        yolo_str_list.append(yolo_label_str)

                output_file_name = output_path + data['FileName'] + '.txt'
                with open(output_file_name ,'w') as f:
                    f.write('\r\n'.join(yolo_str_list))
                    logit('Output file : ' + output_file_name)
                success_count += 1
            else:
                fail_count += 1
                logit('Pic not exist : ' + data['FileName'])
        except:
            logit('Read anno err : ' + anno)
    logit('End anno for each loop. Success : ' + str(success_count) + ' , Fail : ' + str(fail_count)  )

In [256]:
with open(output_path + 'log.txt' ,'w') as f:
    f.write('\r\n'.join(log_list))